# Ben Needs a Friend - LLM Agent for event listings
This is part of the "Ben Needs a Friend" tutorial.  See all the notebooks and materials [here](https://github.com/bpben/ben_friend).

This notebook is intended to be run in Kaggle Notebooks.  Access that version [here](https://www.kaggle.com/code/bpoben/ben-needs-a-friend-llm-agent). 

In this notebook, we set up a simple workflow for an agent to suggest some cool events from the [Boston Calendar](https://www.thebostoncalendar.com/).  This can be run locally or on Kaggle, but requires you to have access to [OpenAI's API](https://openai.com/blog/openai-api). 

In [ ]:
# install requirements (required for Kaggle setup)
%pip install -q langchain langchain_openai
# only if you plan to use the HF setup here
#%pip install -q bitsandbytes accelerate

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import (
    tool, 
    create_react_agent,
    AgentExecutor
)
from langchain_core.prompts import PromptTemplate
from datetime import datetime, date
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser
from langchain.tools.render import render_text_description

# this is for simple scraping tool
import requests
from random import sample
from bs4 import BeautifulSoup

## Setting up the agent
Here are three ways for setting up your LLM.  For more details, see the README! 

1) OpenAI GPT 3.5 - Requires an API key (not free!) You will also need to have an .env file that looks like: `OPENAI_API_KEY=<API key>`

2) Ollama - See the README for setup instructions.  Only tested with Mac M1 OS.

3) Mistral + Kaggle notebook - Designed for use of free GPU, will need installation of packages above.

In [ ]:
# setting up GPT-3.5 configuration
# # using .env file for GPT API key
# from dotenv import load_dotenv
# load_dotenv()
# llm_model = "gpt-3.5-turbo"
# openai = ChatOpenAI(model=llm_model)

# for use in the Kaggle notebook
# you will need to access "Secrets" under the "Add-ons" menu
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("OPENAI_API_KEY")
llm_model = "gpt-3.5-turbo"
openai = ChatOpenAI(api_key=api_key, model=llm_model)

In [ ]:
# running with local Ollama
# see setup instructions - this is different from use in Kaggle
from langchain.llms import Ollama
ollama_instruct_model = 'mistral'

# load pre-trained model
ollama = Ollama(model=ollama_instruct_model)

In [ ]:
# running with Mistral in Kaggle notebook
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig
from langchain_community.llms import HuggingFacePipeline
model_name = '/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1'

# Configure quantization
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

# will use HF's pipeline and LC's wrapping
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                            device_map='auto',# makes use of the GPUs
                                            quantization_config=quantization_config, # for speed/memory
                                            )
pipe = pipeline("text-generation", 
                model=model, tokenizer=tokenizer,
                max_new_tokens=20, # arbitrary, for short answers
               device_map='auto',)

hf_pipe = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# choose your own adventure - for consistency on the llm we'll be using
llm = openai

## Early experiments with agents

If we ask our model to tell us the day, it's not particularly useful.  It has no knowledge of the current date!

In [ ]:
print(llm.invoke('What day is it today?').content)

 But if we design a function and provided the details to the model, it can suggest what steps to take to get the information it needs.

 We'll use LangChain's capability here.  The `@tool` decorator wraps the function that makes it easier to process the text description into input into the model.

In [ ]:
# notice the detailed docstrings - this is to allow the LLM to understand its purpose
# this is a tool decorator for LangChain, it enables it to be parsed 
@tool
def today(text: str) -> str:
    """Returns today's date, use this when you need to get today's date.\
    The input should always be an empty string."""
    return str(date.today())

# render the tool for a prompt (requires input as a list)
render_text_description([today])

#### Try it: Construct a tool-aware prompt
The previous examples with LC should give us some idea how we might plug this tool description into the LLM input so it knows the tool is available.  An example is provided below.

In [ ]:
template = """Answer the following questions as best you can. \
You will need to break your response into steps, each which may use a different tool. \
You have access to the following tools:

{tools}

Question: {input}
"""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm | StrOutputParser()

response = chain.invoke(input={
    "input": "What day is it today?",
    "tools": [today]
    })
print(response)

### Setting up the Reasoning + Act (ReAct) prompt
This is an adapted version of [LangChain's example](https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html).  It provides a structure for the model to generate its outputs as well as placeholders for the tool descriptions and names.

The "scratchpad" is essentially the model's "memory; where all the things its thought and done will be entered.


In [ ]:
# prompt is adapted from LangChain's example: https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html
# action input is required to be a string - otherwise I found it would use function calls as inputs
template = '''You will be a answering questions about upcoming events. \
You will need to break your response into steps, each which may use a different tool. \
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: an input to the action, usually an empty string
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question. 

Think step by step! Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

#### Simple workflow
LC has its own implementation of the ReAct framework.  When you call `create_react_agent`, it will create a pipe with several components.  Let's look at those components:

In [ ]:
# copying this here for reference
@tool
def today(text: str) -> str:
    """Returns today's date, use this when you need to get today's date.
    The input should always be an empty string."""
    return str(date.today())

In [ ]:
tools = [today]
agent = create_react_agent(llm, tools, prompt)
for stage, component in agent:
    if component is not None:
        print(stage)
        if stage=='middle':
            for c in component:
                print(c)
                print('--')
        else:
            print(component.__repr__())

Step by step through the chain above:
- First: The `format_log_to_str` takes the steps that have been performed so far (i.e. thoughts + actions) and formats them into the model input
- Middle
  - First is the prompt, which has "partial variables" corresponding to the tools with passed it.  These will be populated in the tools an tool_names fields in the prompt when the agent runs
  - Then, we have the model itself
    - Note here - there is a "stop" argument - this tells the agent workflow when to stop, so that actions can be parsed 
- Last: The "output parser" which parses the tools and the parameters

All this gets wrappered in an `AgentExecutor`, which controls the flow through the chain (described above) and actually executes the tools, feeding their output back to the LLM.

`verbose`: Enables us to see all the intermediate steps.  
`handle_parsing_errors`: Errors in execution will be fed back to the model to see if it can correct iself.

You'll see in green the text produced by the agent and in blue the output from the function.  

In [ ]:
# run the agent
agent_executor = AgentExecutor(agent=agent, tools=tools, 
                               verbose=True, 
                               handle_parsing_errors=True,
                              max_iterations=5)
print(agent_executor.invoke({"input": "What day is it today?"})['output'])

With the smaller models, you'll see some weird answers.  With GPT 3.5, you'll usually get the workflow you're looking for.

#### Local events workflow
These next functions provide two new tools for our agent; one that provides the current "weekend number" in the month and one that scrapes Boston Calendar for event information.  Most of this is just scraping and formatting Boston Calendar, but the functions with the tool decorators will be provided to the model.

Then, we set up our ReAct agent as before.

A note here - this *usually* works with GPT 3.5.  With other models, it's hit and miss.  That's likely due both to the complexity of the task and the fact that LangChain is...[complicated](https://minimaxir.com/2023/07/langchain-problem/) in how it builds these workflows.

In [ ]:

def parse_for_calendar(text: str) -> str:
    # utility for converting text input to boston calendar URL
    if len(text) == 1:
        # will assume today's month if we're looking at weekend number
        today = datetime.now()
        month = today.month
        day = today.day
        url = f'https://www.thebostoncalendar.com/events?day={day}&month={month}&weekend={text}&year=2024'
    # this will fail if the string provided is not a date
    else:
        try:
            f_date = datetime.strptime(text, '%Y-%M-%d')
            day_of_month = f_date.day
            month = f_date.month
            url = f'https://www.thebostoncalendar.com/events?day={day_of_month}&month={month}&year=2024'
        except ValueError:
            return 
    return url

@tool
def weekend(text: str) -> str:
    """Returns the single-digit weekend number for this weekend, \
    use this for any questions related to the weekend date. \
    The input should always be an empty string, \
    and this function will always return the weekend number."""
    today = datetime.now()
    
    # Calculate the weekend number within the month
    first_day_of_month = today.replace(day=1)
    weekend_number_within_month = (today - first_day_of_month).days // 7 + 1

    return weekend_number_within_month

@tool
def get_events(text: str) -> str:
    """Returns information about local events. \ 
    The input is either a date string in the format YYYY-MM-DD, \
    or it is a single-digit weekend number.\
    This function will return a list where \
    each element contains an event name, date and location as a tuple.\
    This function should be used to provide complete information about events."""
    # use the parsing utility to get a formatted url
    url = parse_for_calendar(text)
    if url is None:
        # give the LLM a useful response
        return f'Input "{text}" is not in the right format - it needs to be a date string or a weekend number'
    response = requests.get(url)     
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract data
    events = soup.find_all('div', class_='info')

    all_events = []
    for event in events:
        title = event.find('h3').text.strip()
        date = event.find('p', class_='time').text.strip()
        location = event.find('p', class_='location').text.strip()
        all_events.append((title, date, location))

    # randomly select a few, provide as list
    # just reducing the amount of tokens being passed to the model, this gives the idea
    choices = sample(all_events, 3)
    
    return choices


In [ ]:
tools = [today, weekend, get_events]
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, 
                               verbose=True, 
                               handle_parsing_errors=True,
                               max_iterations=5)
agent_executor.invoke({"input": "What is going on this weekend?"})

Again, using the smaller models will yield weird results, but GPT is usually able to do this successfully.

We can see here that the output provides three random examples of events coming up this weekend.  One note - if you run this during the weekend, it will return information for the current weekend.

Our "today" tool also enables the agent to get today's events.

In [ ]:
tools = [today, weekend, get_events]
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, 
                               verbose=True, 
                               handle_parsing_errors=True,
                              max_iterations=5)
agent_executor.invoke({"input": "What is going on today?"})

And we can even ask for specific dates!

In [ ]:
agent_executor.invoke({"input": "What is going on 4/28/24?"})

### "Friendly" style
In the above example, we get a sort of generic tone in our response.  Our Friend would never talk like that, would they?


#### Try it: Make the response more friendly
Try and use what you know so far to come up with a way to make the output more in the style of the Friend we designed.

There's a couple ways to do this

1) Edit the ReAct prompt to generate something more friendly

2) Pipe the output of the agent flow into a "styling" prompt.


In [ ]:
# "friendly" version of the ReAct prompt
friend_template = '''Answer the following questions as best you can. \
You will need to break your response into steps, each which may use a different tool. \
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: an input to the action, usually an empty string
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question. 
Output the final answer as if you're having a fun conversation with a good friend.

Think step by step! Begin!

Question: {input}
Thought:{agent_scratchpad}'''

friend_prompt = PromptTemplate.from_template(friend_template)

In [ ]:
agent = create_react_agent(llm, tools, friend_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, 
                               verbose=True, 
                               handle_parsing_errors=True,
                              max_iterations=5)
agent_executor.invoke({"input": "What is going on today?"})

Sometimes this will confuse the flow and you may have less success even with GPT.  With the smaller models, it definitely becomes a challenge.  Another way to get a friendly response would be to do a separate "rewrite" call. 

In [ ]:
# creating a styling prompt
styling_prompt = """Your name is Friend.  You are having a conversation with your close friend Ben. 
You and Ben are sarcastic and poke fun at one another. 
But you care about each other and support one another.

You know the following information:
{events_output}

Ben: {input}
Provide your response:"""

styling_template = PromptTemplate.from_template(styling_prompt)

In [ ]:

styling_agent = {'events_output': agent_executor, 'input': RunnablePassthrough()} | \
styling_template | llm | StrOutputParser()

print(styling_agent.invoke({"input": "What is going on today?"}))